In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
from keras.utils import np_utils,to_categorical
from keras.models import Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPool2D,Flatten,BatchNormalization
from keras.optimizers import Adam,Nadam
from keras import regularizers
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from keras.callbacks import ReduceLROnPlateau

In [ ]:
train = pd.read_csv('/kaggle/input/Kannada-MNIST/train.csv')
test = pd.read_csv('/kaggle/input/Kannada-MNIST/test.csv')
submission = pd.read_csv('/kaggle/input/Kannada-MNIST/sample_submission.csv')
display(train.shape,test.shape)

In [ ]:
Id = test['id']
x_train, x_test, y_train, y_test = train_test_split(train.iloc[:, 1:], train.iloc[:, 0], test_size=0.05)
display(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
x_train, y_train = shuffle(x_train, y_train)
x_train = x_train.values.reshape(-1,28,28,1)/255.0
x_test = x_test.values.reshape(-1,28,28,1)/255.0
y_train = np_utils.to_categorical(y_train,num_classes=10)
y_test = np_utils.to_categorical(y_test,num_classes=10)
training_set=np.concatenate((x_train,x_test),axis=0)
training_labels=np.concatenate((y_train,y_test))

In [ ]:
model=Sequential()
model.add(Conv2D(64,kernel_size=(3,3),input_shape=(28,28,1),activation='relu',padding="same"))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(256,kernel_size=(3,3),activation='relu',padding="same"))
model.add(Conv2D(256,kernel_size=(3,3),activation='relu',padding='same'))
model.add(Conv2D(256,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(512,kernel_size=(3,3),activation='relu',padding="same"))
model.add(Conv2D(512,kernel_size=(3,3),activation='relu',padding='same'))
model.add(Conv2D(512,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPool2D((2,2)))
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))
model.compile(Adam(learning_rate = 0.0007),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [ ]:
model.fit(training_set,training_labels,validation_split=0.1,batch_size=128,epochs=50,callbacks=[learning_rate_reduction])

In [ ]:
test = test.drop(['id'],axis=1)
test = test.values.reshape(test.shape[0],28,28,1)/255.0
FINAL_PREDS = model.predict_classes(test)
submission = pd.DataFrame({ 'id': Id,'label': FINAL_PREDS })
submission.to_csv(path_or_buf ="submission.csv", index=False)
submission.head()